In [2]:
!pip install imgaug


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:0000:0100:01


In [ ]:
import numpy as np
import pandas as pd
import cv2
import imgaug.augmenters as iaa
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from sklearn.model_selection import KFold
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt

# Image augmentation
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-10, 10),
        shear=(-5, 5),
        order=[0, 1],
        mode='reflect'
    )
])

# Create model
def create_model():
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(240, 320, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    angle_output = Dense(1, name='angle_output')(x)
    speed_output = Dense(1, name='speed_output')(x)
    
    model = Model(inputs=base_model.input, outputs=[angle_output, speed_output])
    
    optimizer = Adam(learning_rate=0.001)
    loss = {'angle_output': 'mean_squared_error', 'speed_output': 'mean_squared_error'}
    metrics = {'angle_output': 'mae', 'speed_output': 'mae'}
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9

lr_scheduler = LearningRateScheduler(scheduler)

# Load train dataset
train_df = pd.read_csv('/Users/tim/Desktop/postgraduate/semester 2/machine learning 2/assessment_project/machine-learning-in-science-ii-2023/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'/Users/tim/Desktop/postgraduate/semester 2/machine learning 2/assessment_project/machine-learning-in-science-ii-2023/training_data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    train_images.append(image)
train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])

# Apply image augmentation
train_images = seq(images=train_images)

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
histories = []  
fold = 1

for train_index, val_index in kf.split(train_images):
    X_train, X_val = train_images[train_index], train_images[val_index]
    y_train, y_val = np.array(train_df[['angle', 'speed']].values[train_index]), np.array(train_df[['angle', 'speed']].values[val_index])
    
    model = create_model()
    model_checkpoint = ModelCheckpoint(f"model_MobileNetV2_fold_{fold}_best_val_loss.h5", save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    history = model.fit(X_train, [y_train[:, 0], y_train[:, 1]], epochs=100, batch_size=32, validation_data=(X_val, [y_val[:, 0], y_val[:, 1]]), callbacks=[early_stop, lr_scheduler, model_checkpoint])

    models.append(model)
    histories.append(history)  
    print(f"Finished training fold {fold} with MobileNetV2")

    fold += 1

# Plotting the history of loss and mean absolute error for each fold
for i, history in enumerate(histories):
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Fold {i + 1} Loss')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['angle_output_mae'], label='Training Angle MAE')
    plt.plot(history.history['speed_output_mae'], label='Training Speed MAE')
    plt.plot(history.history['val_angle_output_mae'], label='Validation Angle MAE')
    plt.plot(history.history['val_speed_output_mae'], label='Validation Speed MAE')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Absolute Error')
    plt.legend()
    plt.title(f'Fold {i + 1} Mean Absolute Error')

    plt.show()

# Get the model with the best validation loss
best_model_index = np.argmin([min(history.history['val_loss']) for history in histories])

best_model = models[best_model_index]

# Save the best model
best_model.save('best_model.h5')

Epoch 1/100
345/345 [==============================] - ETA: 0s - loss: 0.1935 - angle_output_loss: 0.1019 - speed_output_loss: 0.0916 - angle_output_mae: 0.1678 - speed_output_mae: 0.2092
Epoch 1: val_loss improved from inf to 44.90345, saving model to model_MobileNetV2_fold_1_best_val_loss.h5
345/345 [==============================] - 387s 1s/step - loss: 0.1935 - angle_output_loss: 0.1019 - speed_output_loss: 0.0916 - angle_output_mae: 0.1678 - speed_output_mae: 0.2092 - val_loss: 44.9035 - val_angle_output_loss: 2.2871 - val_speed_output_loss: 42.6164 - val_angle_output_mae: 1.2632 - val_speed_output_mae: 6.4090 - lr: 0.0010
Epoch 2/100
345/345 [==============================] - ETA: 0s - loss: 0.0560 - angle_output_loss: 0.0206 - speed_output_loss: 0.0353 - angle_output_mae: 0.1112 - speed_output_mae: 0.1280
Epoch 2: val_loss improved from 44.90345 to 3.20174, saving model to model_MobileNetV2_fold_1_best_val_loss.h5
345/345 [==============================] - 388s 1s/step - loss: 0

In [9]:
pip install tensorflow-metal==2.5.0


ERROR: Could not find a version that satisfies the requirement tensorflow-metal==2.5.0 (from versions: 0.5.0, 0.5.1, 0.6.0, 0.7.0, 0.7.1, 0.8.0)
ERROR: No matching distribution found for tensorflow-metal==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [14]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0
